In [1]:
import pandas as pd
from collections import Counter

In [2]:
# df_new_filtered = pd.read_csv("Inappropriate_095_top_vs_one.csv")
df_new_filtered = pd.read_csv("Inappropriate_09_top_vs_one.csv")

In [3]:
df_new_filtered.head()

,INPUT:text,OUTPUT:result,CONFIDENCE:result
0,"Бедный Ниссон, его бесконечных детей то похища...",yes,0.9526
1,Все кто лишился девстенности после 14 - омежки...,yes,0.9972
2,Не знаю сам почему её туда отправили. Не торга...,no,0.9783
3,> Проституция легальнаНелегальное предпринимат...,yes,0.9986
4,"Я бы повстречался с порноактрисой, только я не...",yes,0.9998


In [4]:
df_new_filtered.drop_duplicates(subset=['INPUT:text'],inplace = True)

In [5]:
df_published = pd.read_csv("../appropriateness/Appropriateness.csv")

In [6]:
non_processed_texts_new_filtered = set([txt for txt in df_new_filtered['INPUT:text'].tolist()])
len(non_processed_texts_new_filtered)

124597

In [7]:
df_published.drop_duplicates(subset=['text'],inplace = True)

In [8]:
df_published.head()

,text,inappropriate,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
0,Начальники ДК и нынешнего места работы?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.09
1,"Читаю. Драки, грабежи, ДТП. Изредка убийства, ...",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.18
2,"Допустимо, если информация не нарушает правил ...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.02
3,В больницу при приеме на работу тоже мед комис...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00
4,а зачем ты на проураинские митинги выходил? во...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.40


In [9]:
processed_texts_vers2 = set([txt for txt in df_published['text'].tolist()])
len(processed_texts_vers2)

163312

In [10]:
len(non_processed_texts_new_filtered - processed_texts_vers2)

16503

In [11]:
import re
def get_len(text):
    if len(text.split()) > 5 and len (text)< 250:
        return True
    return None
def is_mostly_russian(text):
    text = str(text)
    russian_letters_count = len(re.findall("[а-яА-Я]",text))
    eng_letters_count = len(re.findall("[a-zA-Z]",text))
    if russian_letters_count > eng_letters_count:
        return True
    return None
def depersonalize(text):
    text = str(text)
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text= re.sub(url_regex, "url", text)
    text = re.sub("id[\d]*",'',text)
    text = re.sub("> ?",'',text)
    text = re.sub("@[\w]*",'',text)
#     text = re.sub("\+[\d]*", "", text)
    text = re.sub("[\d]+", "NUMBER", text)
    text = re.sub(">>", "", text)
#     text = re.sub("[\d]{3,100}", "", text)
    text = re.sub("[\t|\n|\r]", "", text)
#     text = re.sub("[(OP)|url|nickname|phone_number]", "", text)
    return text

In [12]:
processed_texts_new_filtered = set([depersonalize(txt) for txt in non_processed_texts_new_filtered])

In [13]:
a = set([1,2,3])
b = set([2,3,4,5,])
a - b

{1}

In [14]:
len(processed_texts_new_filtered - processed_texts_vers2)

5638

In [15]:
df_new_filtered['text'] = df_new_filtered['INPUT:text'].apply(depersonalize)

In [16]:
df_new_filtered['direct_map'] = df_new_filtered['text'].isin(processed_texts_vers2)

In [17]:
Counter(df_new_filtered['direct_map'])

Counter({True: 118959, False: 5638})

# Там где не нашлись данные

In [ ]:
df_new_filtered_NO_direct_data = df_new_filtered[df_new_filtered['direct_map'] == False]

In [ ]:
# df_new_filtered_NO_direct_data.to_csv("no_direct_data.csv",index = None)
df_new_filtered_NO_direct_data.to_csv("no_direct_data_09.csv",index = None)

In [ ]:
# !pip install transformers

In [18]:
df_new_filtered_NO_direct_data = pd.read_csv("no_direct_data_autolabeled_09.csv")

In [19]:
df_new_filtered_NO_direct_data.head()

,INPUT:text,OUTPUT:result,CONFIDENCE:result,text,direct_map,onerow_multi
0,"Возьми лучше вместо плодовитых ещё декадентов,...",yes,0.9982,"Возьми лучше вместо плодовитых ещё декадентов,...",False,slavery
1,Никогда не брал ее в руки.,no,0.9999,Никогда не брал ее в руки.,False,none
2,"Только для таких дегенератов, как ты, которые ...",yes,0.9998,"Только для таких дегенератов, как ты, которые ...",False,social_injustice
3,"ладнокогда тебе в очко засунут шампанское, ста...",yes,0.9995,"ладнокогда тебе в очко засунут шампанское, ста...",False,pornography
4,Хотя много кто и росияней самоубился бы чтобы ...,yes,0.9974,Хотя много кто и росияней самоубился бы чтобы ...,False,sexism


In [ ]:
# set(df_new_filtered_NO_direct_data['onerow_multi'])

In [20]:
topics = ['offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice']

In [21]:
data_append = []
for  _,el in df_new_filtered_NO_direct_data.iterrows():
    current_topic_list = [0]*len(topics)
    current_topic_list.extend([0,0])
    current_elemen_topics = el['onerow_multi'].split(',')
    for t in current_elemen_topics:
        if t == 'none': continue
        current_topic_list[topics.index(t)] = 1
    data_append.append(current_topic_list)

In [22]:
data_append[:2]

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [23]:
df_append = pd.DataFrame(data = data_append, columns = topics + ['human_labeled', 'toxic_auto'])

In [24]:
df_append.head()

,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [25]:
df_new_filtered_NO_direct_data = pd.concat([df_new_filtered_NO_direct_data,df_append], axis = 1)

In [26]:
df_new_filtered_NO_direct_data.head()

,INPUT:text,OUTPUT:result,CONFIDENCE:result,text,direct_map,onerow_multi,offline_crime,online_crime,drugs,gambling,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
0,"Возьми лучше вместо плодовитых ещё декадентов,...",yes,0.9982,"Возьми лучше вместо плодовитых ещё декадентов,...",False,slavery,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Никогда не брал ее в руки.,no,0.9999,Никогда не брал ее в руки.,False,none,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Только для таких дегенератов, как ты, которые ...",yes,0.9998,"Только для таких дегенератов, как ты, которые ...",False,social_injustice,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"ладнокогда тебе в очко засунут шампанское, ста...",yes,0.9995,"ладнокогда тебе в очко засунут шампанское, ста...",False,pornography,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Хотя много кто и росияней самоубился бы чтобы ...,yes,0.9974,Хотя много кто и росияней самоубился бы чтобы ...,False,sexism,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


# Там где есть просто мапящиеся метаданные

In [27]:
df_new_filtered_with_direct_data = df_new_filtered[df_new_filtered['direct_map'] == True]

In [28]:
df_new_filtered_with_direct_data.head()

,INPUT:text,OUTPUT:result,CONFIDENCE:result,text,direct_map
0,"Бедный Ниссон, его бесконечных детей то похища...",yes,0.9526,"Бедный Ниссон, его бесконечных детей то похища...",True
1,Все кто лишился девстенности после 14 - омежки...,yes,0.9972,Все кто лишился девстенности после NUMBER - ом...,True
2,Не знаю сам почему её туда отправили. Не торга...,no,0.9783,Не знаю сам почему её туда отправили. Не торга...,True
3,> Проституция легальнаНелегальное предпринимат...,yes,0.9986,Проституция легальнаНелегальное предпринимател...,True
4,"Я бы повстречался с порноактрисой, только я не...",yes,0.9998,"Я бы повстречался с порноактрисой, только я не...",True


In [42]:
df_published = df_published.rename(columns = {'text':'publ_text'})

In [43]:
df_new_filtered_with_direct_data_merged = df_new_filtered_with_direct_data.merge(df_published, how = 'inner',
                                                                                 left_on = 'text',
                                                                                right_on = 'publ_text')

In [44]:
len(df_new_filtered_with_direct_data_merged)

118959

In [45]:
df_new_filtered_with_direct_data_merged.columns

Index(['INPUT:text', 'OUTPUT:result', 'CONFIDENCE:result', 'text',
       'direct_map', 'publ_text', 'inappropriate', 'offline_crime',
       'online_crime', 'drugs', 'gambling', 'pornography', 'prostitution',
       'slavery', 'suicide', 'terrorism', 'weapons', 'body_shaming',
       'health_shaming', 'politics', 'racism', 'religion', 'sexual_minorities',
       'sexism', 'social_injustice', 'human_labeled', 'toxic_auto'],
      dtype='object')

In [46]:
df_new_filtered_with_direct_data_merged.sample(2)

,INPUT:text,OUTPUT:result,CONFIDENCE:result,text,direct_map,publ_text,inappropriate,offline_crime,online_crime,drugs,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
94891,"Блин, не прознал, что Вы девушка). Извините, ч...",no,0.9952,"Блин, не прознал, что Вы девушка). Извините, ч...",True,"Блин, не прознал, что Вы девушка). Извините, ч...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.16
43661,За сколько за раз рублей ☼ ☼ ☼ грабил людей Ба...,no,0.9989,За сколько за раз рублей ☼ ☼ ☼ грабил людей Ба...,True,За сколько за раз рублей ☼ ☼ ☼ грабил людей Ба...,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.26


In [48]:
Counter(df_new_filtered_with_direct_data_merged['text'] == df_new_filtered_with_direct_data_merged['publ_text'])

Counter({True: 118959})

# Мерджим

In [49]:
df_new_filtered_with_direct_data_merged.drop(columns=['direct_map','inappropriate','publ_text'],inplace = True)

In [ ]:
# df_new_filtered_NO_direct_data.drop(columns=['direct_map', 'onerow_multi'],inplace = True)

In [51]:
df_new_filtered_with_direct_data_merged.columns 

Index(['INPUT:text', 'OUTPUT:result', 'CONFIDENCE:result', 'text',
       'offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice', 'human_labeled',
       'toxic_auto'],
      dtype='object')

In [57]:
df_new_filtered_NO_direct_data.drop(columns=['direct_map'],inplace = True)

In [58]:
df_new_filtered_NO_direct_data.columns

Index(['INPUT:text', 'OUTPUT:result', 'CONFIDENCE:result', 'text',
       'offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice', 'human_labeled',
       'toxic_auto'],
      dtype='object')

In [59]:
df_new_filtered_with_direct_data_merged.columns == df_new_filtered_NO_direct_data.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [60]:
df_new_filtered_with_direct_data_merged.columns

Index(['INPUT:text', 'OUTPUT:result', 'CONFIDENCE:result', 'text',
       'offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice', 'human_labeled',
       'toxic_auto'],
      dtype='object')

In [ ]:
df_new_filtered_NO_direct_data.columns

In [61]:
df_new_filtered_MERGED = pd.concat([df_new_filtered_with_direct_data_merged,df_new_filtered_NO_direct_data])

In [62]:
Counter(df_new_filtered_MERGED['human_labeled'])

Counter({1: 6025, 0: 118572})

In [63]:
df_new_filtered_MERGED.head()

,INPUT:text,OUTPUT:result,CONFIDENCE:result,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
0,"Бедный Ниссон, его бесконечных детей то похища...",yes,0.9526,"Бедный Ниссон, его бесконечных детей то похища...",1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.31,0.0,0.0,0.0,0.0,0.0,1.00,1,0.59
1,Все кто лишился девстенности после 14 - омежки...,yes,0.9972,Все кто лишился девстенности после NUMBER - ом...,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.00,0,0.58
2,Не знаю сам почему её туда отправили. Не торга...,no,0.9783,Не знаю сам почему её туда отправили. Не торга...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.75,1,0.01
3,> Проституция легальнаНелегальное предпринимат...,yes,0.9986,Проституция легальнаНелегальное предпринимател...,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0,0.16
4,"Я бы повстречался с порноактрисой, только я не...",yes,0.9998,"Я бы повстречался с порноактрисой, только я не...",0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,1,0.64


In [64]:
df_new_filtered_MERGED['INPUT:text'] = df_new_filtered_MERGED['text']
del df_new_filtered_MERGED['text']

In [66]:
conf = []
for i,el in df_new_filtered_MERGED.iterrows():
    if el['OUTPUT:result'] == 'yes':
        conf.append(el['CONFIDENCE:result'])
    else:
        conf.append(1 - float(el['CONFIDENCE:result']))
    

In [67]:
conf[:10]

[0.9526,
 0.9972,
 0.021700000000000053,
 0.9986,
 0.9998,
 0.07809999999999995,
 9.999999999998899e-05,
 0.9998,
 0.9105,
 9.999999999998899e-05]

In [68]:
df_new_filtered_MERGED['CONFIDENCE:result'] = conf
del df_new_filtered_MERGED['OUTPUT:result']
df_new_filtered_MERGED.rename(columns = {'CONFIDENCE:result':'inappropriate'}, inplace = True)

In [70]:
df_new_filtered_MERGED.rename(columns = {'INPUT:text':'text'}, inplace = True)

In [71]:
df_new_filtered_MERGED.head()

,text,inappropriate,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,...,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,human_labeled,toxic_auto
0,"Бедный Ниссон, его бесконечных детей то похища...",0.9526,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.31,0.0,0.0,0.0,0.0,0.0,1.00,1,0.59
1,Все кто лишился девстенности после NUMBER - ом...,0.9972,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.00,0,0.58
2,Не знаю сам почему её туда отправили. Не торга...,0.0217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.75,1,0.01
3,Проституция легальнаНелегальное предпринимател...,0.9986,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0,0.16
4,"Я бы повстречался с порноактрисой, только я не...",0.9998,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,1,0.64


In [72]:
len(df_new_filtered_MERGED)

124597

In [73]:
df_new_filtered_MERGED.to_csv("Inappropriate_09_top_vs_one_with_multi.csv",index = None)

In [ ]:
len(df_new_filtered_MERGED)

In [ ]:
# Считаем пропорции

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
df_new_filtered_MERGED =pd.read_csv("Inappropriate_095_top_vs_one_with_multi.csv")

In [ ]:
topics = ['offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice']

In [ ]:
def get_count(df):
    
    def round_up(val):
        if float(val) > 0.5:
            return 1
        return 0

    df_count = pd.DataFrame()
    for c in topics:
        if c != 'text':
            df_count[c] = df[c].apply(round_up)
            
    counts = []
    names = []
    for col in df_count.columns:
        cnt = sum(df_count[col])
        counts.append(cnt)
        names.append(col)
        
    df_count_fin = pd.DataFrame({'class':names, 'count':counts})
    df_count_fin_sorted = df_count_fin.sort_values('count', ascending = False)
    return df_count_fin_sorted, len(df)

In [ ]:
def plot_df(df, df_len):
    dct = {}
    for  i,d in df.iterrows():
        t,c = d
        dct[t] = c
    same_order_values = [dct[t] for t in topics]
    
        # Generate a normal distribution, center at x=0 and y=5
    x_positions = np.arange(len(topics) )
    y = same_order_values

    # We can set the number of bins with the `bins` kwarg
    plt.bar(x_positions,y, align='center', alpha=0.5)
    plt.xticks(x_positions, topics)
    plt.xticks(rotation = 85, fontsize=14)
    plt.ylabel('count per topic', fontsize=14)
    plt.title("Total сount of samples = {}".format(df_len), fontsize=14)

    plt.show()
#     plt.savefig("multi_count.pdf",format='pdf', bbox_inches='tight')

In [ ]:
df_new_filtered_with_direct_data_merged.head()

In [ ]:
dfr, df_len = get_count(df_new_filtered_MERGED)
plot_df(dfr, df_len)